### 데이터 불러오기

In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Coursework/25-2 Machine Learning/Project/DataPreprocessing_final/closedworld_train.csv")
test = pd.read_csv("/content/drive/MyDrive/Coursework/25-2 Machine Learning/Project/DataPreprocessing_final/closedworld_test.csv")

y_train = train['label']
X_train = train.drop(columns=['label'])

y_test = test['label']
X_test = test.drop(columns=['label'])

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (13300, 26)
y_train shape: (13300,)
X_test shape: (5700, 26)
y_test shape: (5700,)


In [ ]:
# validation set 분리
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

### xgboost 임포트

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier, callback
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, average_precision_score, precision_recall_curve, accuracy_score
)

### 베이스라인 학습

In [ ]:
xgb = XGBClassifier(
    n_estimators=100,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=1.0,
    reg_lambda=1.0,
    reg_alpha=0.0,
    tree_method="hist",
    random_state=42,
    n_jobs=-1,
    eval_metric=["aucpr","auc", "mlogloss"]

)

xgb.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=1
)

[0]	validation_0-aucpr:0.49263	validation_0-auc:0.96874	validation_0-mlogloss:4.28116	validation_1-aucpr:0.41756	validation_1-auc:0.92593	validation_1-mlogloss:4.30696
[1]	validation_0-aucpr:0.58590	validation_0-auc:0.98226	validation_0-mlogloss:4.06361	validation_1-aucpr:0.48334	validation_1-auc:0.95108	validation_1-mlogloss:4.11762
[2]	validation_0-aucpr:0.63004	validation_0-auc:0.98590	validation_0-mlogloss:3.87265	validation_1-aucpr:0.52912	validation_1-auc:0.95744	validation_1-mlogloss:3.95015
[3]	validation_0-aucpr:0.66668	validation_0-auc:0.98810	validation_0-mlogloss:3.70535	validation_1-aucpr:0.55433	validation_1-auc:0.96102	validation_1-mlogloss:3.80580
[4]	validation_0-aucpr:0.68775	validation_0-auc:0.98922	validation_0-mlogloss:3.56072	validation_1-aucpr:0.56870	validation_1-auc:0.96395	validation_1-mlogloss:3.68203
[5]	validation_0-aucpr:0.70615	validation_0-auc:0.99032	validation_0-mlogloss:3.42958	validation_1-aucpr:0.58438	validation_1-auc:0.96510	validation_1-mlogloss:

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False,
              eval_metric=['aucpr', 'auc', 'mlogloss'], feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=1.0, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=-1, num_parallel_tree=None, ...)

### 예측

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
    classification_report
)

In [ ]:
y_pred = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)

# ==== Accuracy ====
accuracy = accuracy_score(y_test, y_pred)

# ==== F1 ====
f1_macro = f1_score(y_test, y_pred, average='macro')
f1_micro = f1_score(y_test, y_pred, average='micro')
f1_weighted = f1_score(y_test, y_pred, average='weighted')

# ==== ROC-AUC ====
roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovr', average='macro')

# ==== PR-AUC ====
pr_auc = average_precision_score(y_test, y_prob, average='macro')

# ==== Classification report ====
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1-macro:", f1_macro)
print("F1-micro:", f1_micro)
print("F1-weighted:", f1_weighted)
print("ROC-AUC:", roc_auc)
print("PR-AUC:", pr_auc)
print(report)

Accuracy: 0.7192982456140351
F1-macro: 0.7171428134484024
F1-micro: 0.7192982456140351
F1-weighted: 0.7171428134484022
ROC-AUC: 0.985357751648625
PR-AUC: 0.7820259639449432
              precision    recall  f1-score   support

           0       0.83      0.57      0.67        60
           1       0.87      0.75      0.80        60
           2       0.84      0.80      0.82        60
           3       0.76      0.87      0.81        60
           4       0.69      0.77      0.72        60
           5       0.58      0.52      0.55        60
           6       0.60      0.88      0.71        60
           7       0.72      0.82      0.77        60
           8       0.73      0.55      0.63        60
           9       0.63      0.55      0.59        60
          10       0.77      0.60      0.67        60
          11       0.90      0.60      0.72        60
          12       0.89      0.83      0.86        60
          13       0.51      0.33      0.40        60
          14    